# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import time

import pprint

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append(".")
#from object_detection.utils import ops as utils_ops

#if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
#    raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')


# Model preparation 

### Variables

In [2]:
#Inference configs
IMAGE_PATH = '/home/jovyan/aidata-volume/AI_Datasets/val2017'
DESTINATION_PATH = '/home/jovyan/'
IMAGE_DIMS = [300, 300]
FIGURE_DIMS = tuple(IMAGE_DIMS.copy())
BATCH_NUM = 200
THRESHOLD = 20

# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03'
MODEL_FILE = MODEL_NAME + '.tar.gz'
INFERENCE_GRAPH_PATH = MODEL_NAME + '/frozen_inference_graph.pb'

#Download url
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

VISUALIZE = True

### Model download

In [3]:
if not os.path.isfile(MODEL_FILE):
    print("Downloading model file")
    opener = urllib.request.URLopener()
    opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
else:
    print("Model file already exists")
    
if not os.path.isfile(INFERENCE_GRAPH_PATH):
    tar_file = tarfile.open(MODEL_FILE)
    for file in tar_file.getmembers():
        file_name = os.path.basename(file.name)
        if 'frozen_inference_graph.pb' in file_name:
            tar_file.extract(file, os.getcwd())
else:
    print('Model file already extracted')

Model file already exists
Model file already extracted


## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [4]:
CATEGORY_INDEX = {}
f = open("object_detection/mscoco_label_map.pbtxt", "r")
for line in f:
    if 'item' in line:
        name = f.readline().split()[1]
        real_id = int(f.readline().split()[1])
        real_name = f.readline().split()[1]
        CATEGORY_INDEX[real_id] =  {'id': real_id, 'name': real_name}

In [5]:
print(CATEGORY_INDEX)

{1: {'id': 1, 'name': 'person'}, 2: {'id': 2, 'name': 'bicycle'}, 3: {'id': 3, 'name': 'car'}, 4: {'id': 4, 'name': 'motorcycle'}, 5: {'id': 5, 'name': 'airplane'}, 6: {'id': 6, 'name': 'bus'}, 7: {'id': 7, 'name': 'train'}, 8: {'id': 8, 'name': 'truck'}, 9: {'id': 9, 'name': 'boat'}, 10: {'id': 10, 'name': 'traffic'}, 11: {'id': 11, 'name': 'fire'}, 13: {'id': 13, 'name': 'stop'}, 14: {'id': 14, 'name': 'parking'}, 15: {'id': 15, 'name': 'bench'}, 16: {'id': 16, 'name': 'bird'}, 17: {'id': 17, 'name': 'cat'}, 18: {'id': 18, 'name': 'dog'}, 19: {'id': 19, 'name': 'horse'}, 20: {'id': 20, 'name': 'sheep'}, 21: {'id': 21, 'name': 'cow'}, 22: {'id': 22, 'name': 'elephant'}, 23: {'id': 23, 'name': 'bear'}, 24: {'id': 24, 'name': 'zebra'}, 25: {'id': 25, 'name': 'giraffe'}, 27: {'id': 27, 'name': 'backpack'}, 28: {'id': 28, 'name': 'umbrella'}, 31: {'id': 31, 'name': 'handbag'}, 32: {'id': 32, 'name': 'tie'}, 33: {'id': 33, 'name': 'suitcase'}, 34: {'id': 34, 'name': 'frisbee'}, 35: {'id': 

### Create dataset as tensors

In [6]:
def create_dataset(path, image_dims, batch_size = 32):
    import pathlib 
    import random 
    print(image_dims)

    
    data_root = pathlib.Path(path)
    image_uris = list(data_root.glob('**/*.jpg'))
    image_uris = [str(image) for image in image_uris]
    
    def preprocess_image(image):

        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize_image_with_pad(image, image_dims[0], image_dims[1])
        #image /= 255  # normalize to [0,1] range

        return image
    
    def load_and_preprocess_image(path):
        
        image = tf.read_file(path)
        
        return preprocess_image(image)
    
    path_ds = tf.data.Dataset.from_tensor_slices(image_uris)
    #AUTOTUNE = tf.data.experimental.AUTOTUNE
    image_ds = path_ds.map(load_and_preprocess_image)
    
    return image_ds

### Create graph and connect tensors

In [7]:
def build_inference_graph(inference_graph_path, d):
    """Loads the inference graph and connects it to the input image.
    Args:
    image_tensor: The input image. uint8 tensor, shape=[1, None, None, 3]
    inference_graph_path: Path to the inference graph with embedded weights
    Returns:
    detected_boxes_tensor: Detected boxes. Float tensor,
        shape=[num_detections, 4]
    detected_scores_tensor: Detected scores. Float tensor,
        shape=[num_detections]
    detected_labels_tensor: Detected labels. Int64 tensor,
        shape=[num_detections]
    """
    #tf.reset_default_graph()
    with tf.gfile.Open(inference_graph_path, 'rb') as graph_def_file:
        graph_content = graph_def_file.read()
        graph_def = tf.GraphDef()
        graph_def.MergeFromString(graph_content)


    # Then, we can use again a convenient built-in function to import a graph_def into the 
    # current default Graph
    with tf.Graph().as_default() as graph:
        with tf.device(d):
            tf.import_graph_def(
                graph_def,
                name = ''
            )

    #graph = tf.get_default_graph()
    tensor_dict = {}
    #image_tensor = None
    for key in ['num_detections', 'detection_boxes', 'detection_scores', 'detection_classes']:
        tensor_name = key + ':0'
        try:
            tensor_dict[key] = graph.get_tensor_by_name(tensor_name)
        except:
            print("Something went horribly wrong when loading graph tensors")
            pass


    image_tensor = graph.get_tensor_by_name('image_tensor:0')
    tf.reset_default_graph()
    return {'out': tensor_dict, 'in': image_tensor}, graph

# Threaded visualization

In [8]:
from threading import Thread
from object_detection.utils import visualization_utils as vis_util

vis_threads = []

def threaded_function(image_np, args_dict, counter):
    boxes = args_dict['detection_boxes']
    scores = args_dict['detection_scores']
    classes = np.array(args_dict['detection_classes'], np.int16)
    
    for i in range(len(image_np)):
        new_img = np.array(image_np[i], np.uint8)
        vis_util.visualize_boxes_and_labels_on_image_array(
          new_img,
          boxes[i],
          classes[i],
          scores[i],
          CATEGORY_INDEX,
          use_normalized_coordinates=True,
          line_thickness=8)
        plt.figure(figsize=FIGURE_DIMS)
        #plt.imsave(DESTINATION_PATH + '/{}_{}.jpg'.format(str(counter), str(i)), new_img)
        plt.close()

In [9]:
def threaded_inference(device, tensor_dict, image_tensor, graph, image, counter):
    print(image_tensor)  <y
    with tf.Session(graph = graph) as sess:
        output_dict = sess.run(tensor_dict, feed_dict = {image_tensor: image})

        if VISUALIZE:
                thread = Thread(target = threaded_function, args = (image, output_dict, counter))
                thread.start()
                vis_threads.append(thread)
        

# Actual Inference

In [10]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    #return [x.name for x in local_device_protos if (x.device_type == 'GPU' or x.device_type == 'CPU')]
    return [x.name for x in local_device_protos if (x.device_type == 'GPU')]

In [11]:
print(get_available_gpus())

['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7']


In [12]:

ds = create_dataset(IMAGE_PATH, IMAGE_DIMS)
ds = ds.prefetch(BATCH_NUM*2).batch(BATCH_NUM)
it = ds.make_one_shot_iterator()

# This is needed to display the images.
%matplotlib inline

devices = get_available_gpus()

tf.logging.info('Reading graph and building model specified for the different cluster devices')

tf_dict = []
tg_dict = []
n = 0
for d in devices:
    print(d + " loading graph done!")
    tensors, graph = build_inference_graph(INFERENCE_GRAPH_PATH, d)
    tf_dict.append(tensors)
    tg_dict.append(graph)

#image_it = it.get_next()

#threads = []

i = 1
j = 1

try:
    while True:
        threads = []
        print('Step: ' + str(i))
        for idx in range(len(tf_dict)):
            loc_image = []
            #with tf.Session() as sess:
            #        sess.run(tf.global_variables_initializer())
            #        loc_image = sess.run(image_it)
            loc_image = np.arange(200*300*300*3).reshape((200, 300, 300, 3))
            thread = Thread(target = threaded_inference, 
                            args = (devices[idx], tf_dict[idx]['out'], tf_dict[idx]['in'], tg_dict[idx], loc_image, i))
            i += 1
            thread.start()
            threads.append(thread)

        for t in threads:
            t.join()


except tf.errors.OutOfRangeError:
    tf.logging.info('Finished processing records')
    for t in vis_threads:
        t.join()


[300, 300]
INFO:tensorflow:Reading graph and building model specified for the different cluster devices
/device:GPU:0 loading graph done!
/device:GPU:1 loading graph done!
/device:GPU:2 loading graph done!
/device:GPU:3 loading graph done!
/device:GPU:4 loading graph done!
/device:GPU:5 loading graph done!
/device:GPU:6 loading graph done!
/device:GPU:7 loading graph done!
Step: 1
Tensor("image_tensor:0", shape=(?, ?, ?, 3), dtype=uint8, device=/device:GPU:0)
['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7']
Tensor("image_tensor:0", shape=(?, ?, ?, 3), dtype=uint8, device=/device:GPU:1)
['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7']
Tensor("image_tensor:0", shape=(?, ?, ?, 3), dtype=uint8, device=/device:GPU:2)
['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5',

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1334, in _do_call
    return fn(*args)
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1317, in _run_fn
    self._extend_graph()
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1352, in _extend_graph
    tf_session.ExtendSession(self._session)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Cannot assign a device for operation Preprocessor/map/TensorArray_2: Could not satisfy explicit device specification '/device:GPU:0' because no supported kernel for GPU devices is available.
Colocation Debug Info:
Colocation group had the following types and devices: 
TensorArrayGatherV3: GPU CPU XLA_CPU XLA_GPU 
TensorArrayWriteV3: CPU XLA_CPU XLA_GPU 
TensorArraySizeV3: GPU CPU XLA_CPU XLA_GPU 
Enter: GPU CPU XLA_CPU XLA_GPU 
TensorArrayV3: CPU XLA_

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1334, in _do_call
    return fn(*args)
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1317, in _run_fn
    self._extend_graph()
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1352, in _extend_graph
    tf_session.ExtendSession(self._session)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Cannot assign a device for operation Preprocessor/map/TensorArray_2: Could not satisfy explicit device specification '/device:GPU:1' because no supported kernel for GPU devices is available.
Colocation Debug Info:
Colocation group had the following types and devices: 
TensorArrayGatherV3: GPU CPU XLA_CPU XLA_GPU 
TensorArrayWriteV3: CPU XLA_CPU XLA_GPU 
TensorArraySizeV3: GPU CPU XLA_CPU XLA_GPU 
Enter: GPU CPU XLA_CPU XLA_GPU 
TensorArrayV3: CPU XLA_

Tensor("image_tensor:0", shape=(?, ?, ?, 3), dtype=uint8, device=/device:GPU:5)
['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7']
Tensor("image_tensor:0", shape=(?, ?, ?, 3), dtype=uint8, device=/device:GPU:6)
['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7']


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1334, in _do_call
    return fn(*args)
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1317, in _run_fn
    self._extend_graph()
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1352, in _extend_graph
    tf_session.ExtendSession(self._session)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Cannot assign a device for operation Preprocessor/map/TensorArray_2: Could not satisfy explicit device specification '/device:GPU:2' because no supported kernel for GPU devices is available.
Colocation Debug Info:
Colocation group had the following types and devices: 
TensorArrayGatherV3: GPU CPU XLA_CPU XLA_GPU 
TensorArrayWriteV3: CPU XLA_CPU XLA_GPU 
TensorArraySizeV3: GPU CPU XLA_CPU XLA_GPU 
Enter: GPU CPU XLA_CPU XLA_GPU 
TensorArrayV3: CPU XLA_

...

Exception in thread Thread-68:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1334, in _do_call
    return fn(*args)
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1317, in _run_fn
    self._extend_graph()
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1352, in _extend_graph
    tf_session.ExtendSession(self._session)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Cannot assign a device for operation Preprocessor/map/TensorArray_2: Could not satisfy explicit device specification '/device:GPU:0' because no supported kernel for GPU devices is available.
Colocation Debug Info:
Colocation group had the following types and devices: 
TensorArrayGatherV3: GPU CPU XLA_CPU XLA_GPU 
TensorArrayWriteV3: CPU XLA_CPU XLA_GPU 
TensorArraySizeV3: GPU CPU XLA_CPU XLA_GPU 
Enter: GPU CPU XLA_CPU XLA_GPU 
TensorArrayV3: CPU XLA

Tensor("image_tensor:0", shape=(?, ?, ?, 3), dtype=uint8, device=/device:GPU:3)
['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7']
Tensor("image_tensor:0", shape=(?, ?, ?, 3), dtype=uint8, device=/device:GPU:4)
['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7']


Exception in thread Thread-70:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1334, in _do_call
    return fn(*args)
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1317, in _run_fn
    self._extend_graph()
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1352, in _extend_graph
    tf_session.ExtendSession(self._session)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Cannot assign a device for operation Preprocessor/map/TensorArray_2: Could not satisfy explicit device specification '/device:GPU:2' because no supported kernel for GPU devices is available.
Colocation Debug Info:
Colocation group had the following types and devices: 
TensorArrayGatherV3: GPU CPU XLA_CPU XLA_GPU 
TensorArrayWriteV3: CPU XLA_CPU XLA_GPU 
TensorArraySizeV3: GPU CPU XLA_CPU XLA_GPU 
Enter: GPU CPU XLA_CPU XLA_GPU 
TensorArrayV3: CPU XLA

Exception in thread Thread-72:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1334, in _do_call
    return fn(*args)
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1317, in _run_fn
    self._extend_graph()
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1352, in _extend_graph
    tf_session.ExtendSession(self._session)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Cannot assign a device for operation Preprocessor/map/TensorArray_2: Could not satisfy explicit device specification '/device:GPU:4' because no supported kernel for GPU devices is available.
Colocation Debug Info:
Colocation group had the following types and devices: 
TensorArrayGatherV3: GPU CPU XLA_CPU XLA_GPU 
TensorArrayWriteV3: CPU XLA_CPU XLA_GPU 
TensorArraySizeV3: GPU CPU XLA_CPU XLA_GPU 
Enter: GPU CPU XLA_CPU XLA_GPU 
TensorArrayV3: CPU XLA

KeyboardInterrupt: 

ds = create_dataset(IMAGE_PATH, IMAGE_DIMS)
ds = ds.prefetch(BATCH_NUM*2).batch(BATCH_NUM)
it = ds.make_one_shot_iterator()

# This is needed to display the images.
%matplotlib inline

#config=tf.ConfigProto(log_device_placement=True)


#tf.logging.info('Reading input from files and connecting to graph')
#image_tensor = build_input(image_path)

with tf.Session() as sess:
    devices = get_available_gpus()
    sess.run(tf.global_variables_initializer())
    tf.logging.info('Reading graph and building model specified for the different cluster devices')

    tf_dict = []
    tg_dict = []
    for d in devices:
        print(d + " loading graph done!")
        tensors, graph = build_inference_graph(INFERENCE_GRAPH_PATH, d)
        tf_dict.append(tensors)
        tg_dict.append(graph)
        
    print(tg_dict)
    image_it = it.get_next()

    threads = []
    

    i = 1
    j = 1
    try:
        while True:
            print('Step: ' + str(i))
            start = time.time()

            output_multi_device = []
            input_feed_dict = {}
            images = []
            '''
            for idx in tf_dict:

                output_multi_device.append(idx['out'])
                loc_image = sess.run(image_it)
                print(idx['in'])
                input_feed_dict[idx['in']] = loc_image
                images.append(loc_image)

            outputs = sess.run(output_multi_device, input_feed_dict)
            #pprint.pprint(outputs)

            if VISUALIZE:
                for count in range(len(tf_dict)):
                    thread = Thread(target = threaded_function, args = (images[count], outputs[count], i, count))
                    thread.start()
                    threads.append(thread)
            i += 1
            '''
            threads = []
            for idx in range(len(tf_dict)):
                loc_image = []
                #with tf.Session() as sess:
                #        sess.run(tf.global_variables_initializer())
                #        loc_image = sess.run(image_it)
                loc_image = np.arange(200*300*300*3).reshape((200, 300, 300, 3))
                thread = Thread(target = threaded_inference, 
                                args = (devices[idx], tf_dict[idx]['out'], tf_dict[idx]['in'], tg_dict[idx], loc_image, i))
                i += 1
                thread.start()
                threads.append(thread)

            for t in threads:
                t.join()
            
    except tf.errors.OutOfRangeError:
        tf.logging.info('Finished processing records')
        for t in vis_threads:
            t.join()

In [ ]:
print(tf_dict[2]['out']['num_detections'])
print(tf_dict[2]['in'])

In [ ]:
print(tensor_dict)